In [1]:
from util import *

['Solarize_Light2', '_classic_test_patch', '_mpl-gallery', '_mpl-gallery-nogrid', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn-v0_8', 'seaborn-v0_8-bright', 'seaborn-v0_8-colorblind', 'seaborn-v0_8-dark', 'seaborn-v0_8-dark-palette', 'seaborn-v0_8-darkgrid', 'seaborn-v0_8-deep', 'seaborn-v0_8-muted', 'seaborn-v0_8-notebook', 'seaborn-v0_8-paper', 'seaborn-v0_8-pastel', 'seaborn-v0_8-poster', 'seaborn-v0_8-talk', 'seaborn-v0_8-ticks', 'seaborn-v0_8-white', 'seaborn-v0_8-whitegrid', 'tableau-colorblind10']
Connected DB CINS_SMY - conn
Connected oraDW_ANALYTICS - conn_aly


# Analyse Behavior (update continuously)

In [2]:
# cohort06 = pd.read_pickle('data/COHORT_06')
cohort07 = pd.read_pickle('data/COHORT_07')
cohort08 = pd.read_pickle('data/COHORT_08')
cohort09 = pd.read_pickle('data/COHORT_09')
cohort10 = pd.read_pickle('data/COHORT_10')
cohort11 = pd.read_pickle('data/COHORT_11')
# cohort12 = pd.read_pickle('data/COHORT_12')

In [3]:
k = 30000

Deploy lead Jun 2023

In [4]:
cust = []
# x6 = cohort06.sort_values(by='SCORE', ascending=False)[:k]
# cust.extend(x6['CUSTOMER_CDE'].tolist())
x7 = cohort07[~cohort07['CUSTOMER_CDE'].isin(cust)].sort_values(by='SCORE', ascending=False)[:k]
cust.extend(x7['CUSTOMER_CDE'].tolist())
x8 = cohort08[~cohort08['CUSTOMER_CDE'].isin(cust)].sort_values(by='SCORE', ascending=False)[:k]
cust.extend(x8['CUSTOMER_CDE'].tolist())
x9 = cohort09[~cohort09['CUSTOMER_CDE'].isin(cust)].sort_values(by='SCORE', ascending=False)[:k]
cust.extend(x9['CUSTOMER_CDE'].tolist())
x10 = cohort10[~cohort10['CUSTOMER_CDE'].isin(cust)].sort_values(by='SCORE', ascending=False)[:k]
cust.extend(x10['CUSTOMER_CDE'].tolist())
x11 = cohort11[~cohort11['CUSTOMER_CDE'].isin(cust)].sort_values(by='SCORE', ascending=False)[:k]
cust.extend(x11['CUSTOMER_CDE'].tolist())
# x12 = cohort12[~cohort12['CUSTOMER_CDE'].isin(cust)].sort_values(by='SCORE', ascending=False)[:k]
# cust.extend(x12['CUSTOMER_CDE'].tolist())
print(len(set(cust)))

150000


In [5]:
# len(x6), 
len(x7), len(x8), len(x9), len(x10), len(x11), 
# len(x12)

(30000, 30000, 30000, 30000, 30000)

In [6]:
# set(x6['CUSTOMER_CDE'].tolist()) & set(x7['CUSTOMER_CDE'].tolist()), set(x6['CUSTOMER_CDE'].tolist()) & set(x8['CUSTOMER_CDE'].tolist()), set(x6['CUSTOMER_CDE'].tolist()) & set(x9['CUSTOMER_CDE'].tolist()), set(x6['CUSTOMER_CDE'].tolist()) & set(x12['CUSTOMER_CDE'].tolist()), set(x7['CUSTOMER_CDE'].tolist()) & set(x8['CUSTOMER_CDE'].tolist()), set(x7['CUSTOMER_CDE'].tolist()) & set(x9['CUSTOMER_CDE'].tolist()), set(x7['CUSTOMER_CDE'].tolist()) & set(x10['CUSTOMER_CDE'].tolist()), set(x7['CUSTOMER_CDE'].tolist()) & set(x11['CUSTOMER_CDE'].tolist())

In [7]:
# x6['CUST_STT'].value_counts(), 
x7['CUST_STT'].value_counts(), x8['CUST_STT'].value_counts(), x9['CUST_STT'].value_counts(), x10['CUST_STT'].value_counts(), x11['CUST_STT'].value_counts(), 
# x12['CUST_STT'].value_counts()

(1    29760
 0      240
 Name: CUST_STT, dtype: int64,
 1    28472
 0     1528
 Name: CUST_STT, dtype: int64,
 1    27454
 0     2546
 Name: CUST_STT, dtype: int64,
 1    28034
 0     1966
 Name: CUST_STT, dtype: int64,
 1    28649
 0     1351
 Name: CUST_STT, dtype: int64)

In [8]:
def analyze_cohort(cohort):
    mob = cohort['MOB'].tolist()[0]
    len_cohort = len(cohort)
    data = []
    num_react = len(cohort[cohort['CUST_STT'] == 2])
    rpt_dt = cohort['RPT_DT'].tolist()[0]
    cohort = cohort[cohort['CUST_STT'] != 2]
    len_not_react = len(cohort)
    data.append((rpt_dt,num_react, len_not_react))
    if len(cohort) > 0:
        for i in range(1,10):
            if len(cohort) > 0:
                lead_mth = f'CUST_STT_LEAD{i}'
                num_react = len(cohort[cohort[lead_mth] == 2])
                rpt_dt = cohort[f'RPT_DT_LEAD{i}'].tolist()[0]
                cohort = cohort[cohort[lead_mth] != 2]
                len_not_react = len(cohort)
                data.append((rpt_dt,num_react, len_not_react))
    res_df = pd.DataFrame(data=data, columns=['RPT_DT', 'NUM_REACT', 'NUM_NOT_REACT'])
    res_df['PERC_REACT_PREV_MOB'] = res_df['NUM_REACT'].shift(-1) / res_df['NUM_NOT_REACT']
    res_df['PERC_REACT_PREV_MOB'] = res_df['PERC_REACT_PREV_MOB'].shift()
    res_df['CUMSUM'] = res_df['NUM_REACT'].cumsum()
    res_df['PERC'] = res_df['CUMSUM'] / len_cohort
    res_df = res_df[res_df['RPT_DT'].notnull()]
    res_df = res_df.reset_index()
    # ax = sns.barplot(res_df, x='RPT_DT', y='NUM_REACT')
    # sns.lineplot(res_df['CUMSUM'], ax=ax)
    # plt.show()
    res_df['MOB'] = mob
    res_df = res_df.set_index('MOB')
    return res_df

In [9]:
# x6['MOB'] = '01-06-2023'
x7['MOB'] = '01-07-2023'
x8['MOB'] = '01-08-2023'
x9['MOB'] = '01-09-2023'
x10['MOB'] = '01-10-2023'
x11['MOB'] = '01-11-2023'
# x12['MOB'] = '01-12-2023'

In [10]:
# t = analyze_cohort(x6)
t = analyze_cohort(x7)
for x in [x8, x9, x10, x11]:
    t = pd.concat([t, analyze_cohort(x)])

In [11]:
t

,index,RPT_DT,NUM_REACT,NUM_NOT_REACT,PERC_REACT_PREV_MOB,CUMSUM,PERC
MOB,,,,,,,
01-07-2023,0,2023-06-30,0,30000,NaN,0,0.000000
01-07-2023,1,2023-07-31,11846,18154,0.394867,11846,0.394867
01-07-2023,2,2023-08-31,1662,16492,0.091550,13508,0.450267
01-07-2023,3,2023-09-30,913,15579,0.055360,14421,0.480700
01-07-2023,4,2023-10-31,892,14687,0.057257,15313,0.510433
01-07-2023,5,2023-11-30,1591,13096,0.108327,16904,0.563467
01-07-2023,6,2023-12-31,220,12876,0.016799,17124,0.570800
01-08-2023,0,2023-07-31,0,30000,NaN,0,0.000000
01-08-2023,1,2023-08-31,9357,20643,0.311900,9357,0.311900


In [12]:
t = analyze_cohort(x7)[['PERC']]
t.rename(columns={'PERC': 'MoB-7'}, inplace=True)
t = t.reset_index(drop=True)
p = 8
for x in [x8, x9, x10, x11]:
    z = analyze_cohort(x)[['PERC']]
    z.rename(columns={'PERC': f'Mob-{p}'}, inplace=True)
    z = z.reset_index(drop=True)
    t = pd.concat([t, z], axis=1)
    p += 1

In [13]:
t.T

,0,1,2,3,4,5,6
MoB-7,0.0,0.394867,0.450267,0.480700,0.510433,0.563467,0.5708
Mob-8,0.0,0.311900,0.354367,0.387933,0.439667,0.448200,NaN
Mob-9,0.0,0.313667,0.363567,0.423533,0.434100,NaN,NaN
Mob-10,0.0,0.352133,0.432300,0.446267,NaN,NaN,NaN
Mob-11,0.0,0.394233,0.426400,NaN,NaN,NaN,NaN
